# Data Cleaning and Domain Classification Pipeline

This notebook cleans and enriches the merged Semantic Scholar papers database by:
1. Filtering papers with complete metadata (title, abstract, publication date)
2. Classifying papers by domain (AI, Physics, Psychology) using hierarchical matching
3. Extracting primary topic information from OpenAlex domain databases
4. Optimizing storage for the final cleaned dataset
5. Uploading the final database to Hugging Face Hub for public access

**Input Databases:**
- `S2_papers.db` - Merged Semantic Scholar papers with abstracts (from `merge_papers_and_abstracts.ipynb`)
- `openalex_ai.db` - OpenAlex AI papers
- `openalex_physics.db` - OpenAlex Physics papers  
- `openalex_psych.db` - OpenAlex Psychology papers

**Output:**
- `S2_papers_cleaned.db` - Cleaned and domain-classified papers (temporary)
- `S2_papers_cleaned_optimized.db` - Storage-optimized final database
- Hugging Face Dataset: [`lalit3c/S2_CS_PHY_PYSCH_papers`](https://huggingface.co/datasets/lalit3c/S2_CS_PHY_PYSCH_papers)

In [14]:
import pandas as pd
import duckdb
import os
import time

## 1. Setup and Database Connections

Import required libraries and establish connections to all source and target databases.

In [15]:
DB_DIR = 'db'
if not os.path.exists(DB_DIR):
    os.makedirs(DB_DIR)

S2_PAPERS_DB_FILENAME = 'S2_papers.db'
PHYSICS_DB_FILENAME = 'openalex_physics.db'
AI_DB_FILENAME = 'openalex_ai.db'
PSYCH_PAPERS_DB_FILENAME = 'openalex_psych.db'
CLEANED_UP_S2_PAPERS_DB_FILENAME = 'S2_papers_cleaned.db'
    
papers_w_abstracts = duckdb.connect(os.path.join(DB_DIR, S2_PAPERS_DB_FILENAME))
physics_papers = duckdb.connect(os.path.join(DB_DIR, PHYSICS_DB_FILENAME))
ai_papers = duckdb.connect(os.path.join(DB_DIR, AI_DB_FILENAME))
psych_papers = duckdb.connect(os.path.join(DB_DIR, PSYCH_PAPERS_DB_FILENAME))
cleaned_db = duckdb.connect(os.path.join(DB_DIR, CLEANED_UP_S2_PAPERS_DB_FILENAME))

cleaned_db.execute(f""" ATTACH '{os.path.join(DB_DIR, PHYSICS_DB_FILENAME)}' AS physics_db; """)
cleaned_db.execute(f""" ATTACH '{os.path.join(DB_DIR, AI_DB_FILENAME)}' AS ai_db; """)
cleaned_db.execute(f""" ATTACH '{os.path.join(DB_DIR, PSYCH_PAPERS_DB_FILENAME)}' AS psych_db; """)
cleaned_db.execute(f""" ATTACH '{os.path.join(DB_DIR, S2_PAPERS_DB_FILENAME)}' AS papers_w_abstracts_db; """)

## 2. Data Exploration

Examine sample records from each domain database and the merged papers database to understand schema and data quality.

In [10]:
# 3 papers form each db for testing
physics_sample = physics_papers.execute("SELECT * FROM works LIMIT 3").df()
ai_sample = ai_papers.execute("SELECT * FROM works LIMIT 3").df()
psych_sample = psych_papers.execute("SELECT * FROM works LIMIT 3").df()
samples_df = pd.concat([physics_sample, ai_sample, psych_sample], ignore_index=True)
samples_df

,id,title,doi,publication_date,primary_topic,version,fwci,citation_count,mag_id
0,W2084591555,"Sea breeze: Structure, forecasting, and impacts",https://doi.org/10.1029/2003rg000124,2003-09-01,Meteorological Phenomena and Simulations,publishedVersion,2.740224,564,2084591555
1,W2155944712,Statistical modeling of tornado intensity dist...,https://doi.org/10.1016/s0169-8095(03)00050-4,2003-06-30,Meteorological Phenomena and Simulations,publishedVersion,1.992890,84,2155944712
2,W2004967150,An Investigation of Turbulence Generation Mech...,https://doi.org/10.1175/1520-0469(2003)60<1297...,2003-04-30,Meteorological Phenomena and Simulations,publishedVersion,5.729560,174,2004967150
3,W2147096324,AnalogySpace: reducing the dimensionality of c...,None,2008-07-13,Topic Modeling,submittedVersion,22.747139,171,2147096324
4,W2143930661,A Topic Modeling Approach and Its Integration ...,https://doi.org/10.1109/icdm.2008.71,2008-12-01,Topic Modeling,publishedVersion,9.976816,141,2143930661
5,W2097433297,Generating Impact-Based Summaries for Scientif...,None,2008-06-01,Topic Modeling,submittedVersion,5.587017,90,2097433297
6,W1998234914,Association of psychosocial risk factors with ...,https://doi.org/10.1016/s0140-6736(04)17019-0,2004-09-01,Cardiac Health and Mental Health,publishedVersion,46.289101,2363,1998234914
7,W2047319975,The measurement of effort–reward imbalance at ...,https://doi.org/10.1016/s0277-9536(03)00351-4,2003-09-03,Workplace Health and Well-being,publishedVersion,65.924998,2180,2047319975
8,W2157229623,Lost Productive Time and Cost Due to Common Pa...,https://doi.org/10.1001/jama.290.18.2443,2003-11-11,Musculoskeletal pain and rehabilitation,publishedVersion,27.947113,1564,2157229623


In [11]:
s2_papers_sample = papers_w_abstracts.execute("SELECT * FROM papers_with_abstracts LIMIT 5").df()
s2_papers_sample

,corpusid,title,publication_date,citation_count,influential_citation_count,field_of_study,externalids,abstract,usable,primary_topic,domain,is_physics,is_ai,is_psych,ai_match_method,physics_match_method,psych_match_method
0,214745815,Using Expectancy Violations Theory to Understa...,2020-03-23,10,0,"Computer Science, Psychology","{""Medline"": null, ""MAG"": ""3014184172"", ""ACL"": ...",As robots are increasingly placed in direct in...,True,Psychology of Social Influence,Psychology,False,False,True,None,None,title
1,67820229,A Novel Polarization Independent Wideband Circ...,2018-09-14,9,0,"Engineering, Mathematics, Physics","{""Medline"": null, ""MAG"": ""2890228398"", ""ACL"": ...","In this paper, a polarization independent wide...",False,None,None,False,False,False,None,None,None
2,220287957,The Impact of Chronic Illness on Patients.,2020-07-01,4,0,"Medicine, Psychology","{""Medline"": ""32604258v1"", ""MAG"": null, ""ACL"": ...",The purpose of this study was to assess the im...,False,None,None,False,False,False,None,None,None
3,115843305,Probabilistic and multi‐objective approach for...,2019-01-30,6,0,"Computer Science, Engineering, Environmental S...","{""Medline"": null, ""MAG"": ""2914396121"", ""ACL"": ...",This study aims to present an architecture for...,False,None,None,False,False,False,None,None,None
4,234439708,M/G/1 Preemptive Priority Queues With Finite a...,2020-12-30,2,0,Computer Science,"{""Medline"": null, ""MAG"": ""3118546099"", ""ACL"": ...","Recently, M/G/1 priority queues with a finite ...",False,None,None,False,False,False,None,None,None


In [12]:
COLUMNS_TO_KEEP = ['corpusid', 'title', 'abstract', 'externalids', 'publication_date', 'citation_count', 'influential_citation_count']

## 3. Data Quality Filtering

Filter papers to retain only those with complete metadata: non-empty title, abstract, and publication date. Papers published before 2000 are excluded.

In [13]:
papers_w_abstracts.execute(f"""SELECT count(*) FROM papers_with_abstracts 
    WHERE  title IS NOT NULL AND title != '' AND
        abstract IS NOT NULL AND abstract != '' AND
        publication_date IS NOT NULL AND publication_date != ''
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count_star()
0,7579124


In [14]:
# count where publication_date is after 2000-01-01
papers_w_abstracts.execute(f"""SELECT count(*) FROM papers_with_abstracts 
    WHERE  title IS NOT NULL AND title != '' AND
        abstract IS NOT NULL AND abstract != '' AND
        publication_date IS NOT NULL AND publication_date != '' AND
        publication_date >= '2000-01-01'
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count_star()
0,7411227


In [15]:
COLUMNS_TO_KEEP_STR = ', '.join(COLUMNS_TO_KEEP)

In [16]:
# copy rows from S2 db where title, abstract, publication_date are not null/empty and publication_date is after 2000-01-01
START_DATE = '2000-01-01'
cleaned_db.execute(f"""
    CREATE TABLE papers_with_abstracts AS 
    SELECT {COLUMNS_TO_KEEP_STR} FROM papers_w_abstracts_db.papers_with_abstracts 
    WHERE  title IS NOT NULL AND title != '' AND
        abstract IS NOT NULL AND abstract != '' AND
        publication_date IS NOT NULL AND publication_date != ''
        AND publication_date >= '{START_DATE}';
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,7411227


In [17]:
# samples from cleaned db
cleaned_db.execute("SELECT * FROM papers_with_abstracts LIMIT 5").df()

,corpusid,title,abstract,externalids,publication_date,citation_count,influential_citation_count
0,214745815,Using Expectancy Violations Theory to Understa...,As robots are increasingly placed in direct in...,"{""Medline"": null, ""MAG"": ""3014184172"", ""ACL"": ...",2020-03-23,10,0
1,67820229,A Novel Polarization Independent Wideband Circ...,"In this paper, a polarization independent wide...","{""Medline"": null, ""MAG"": ""2890228398"", ""ACL"": ...",2018-09-14,9,0
2,220287957,The Impact of Chronic Illness on Patients.,The purpose of this study was to assess the im...,"{""Medline"": ""32604258v1"", ""MAG"": null, ""ACL"": ...",2020-07-01,4,0
3,115843305,Probabilistic and multi‐objective approach for...,This study aims to present an architecture for...,"{""Medline"": null, ""MAG"": ""2914396121"", ""ACL"": ...",2019-01-30,6,0
4,234439708,M/G/1 Preemptive Priority Queues With Finite a...,"Recently, M/G/1 priority queues with a finite ...","{""Medline"": null, ""MAG"": ""3118546099"", ""ACL"": ...",2020-12-30,2,0


In [ ]:
# Add columns to CCleaned db papers_w_abstracts with hierarchical matching (Title -> DOI -> MAG ID)
_tik = time.time()
cleaned_db.execute("ALTER TABLE papers_with_abstracts ADD COLUMN IF NOT EXISTS usable BOOLEAN")
cleaned_db.execute("ALTER TABLE papers_with_abstracts ADD COLUMN IF NOT EXISTS primary_topic VARCHAR")
cleaned_db.execute("ALTER TABLE papers_with_abstracts ADD COLUMN IF NOT EXISTS is_physics BOOLEAN")
cleaned_db.execute("ALTER TABLE papers_with_abstracts ADD COLUMN IF NOT EXISTS is_ai BOOLEAN")
cleaned_db.execute("ALTER TABLE papers_with_abstracts ADD COLUMN IF NOT EXISTS is_psych BOOLEAN")
cleaned_db.execute("""ALTER TABLE papers_with_abstracts ADD COLUMN IF NOT EXISTS match_method VARCHAR""")
print(f"Added columns in {time.time() - _tik:.2f} seconds.")


Added columns in 0.31 seconds.


## 4. Domain Classification

Classify papers into research domains (AI, Physics, Psychology) by matching against OpenAlex domain databases. A hierarchical matching strategy is employed:

1. **Title matching** - Exact title match (highest precision)
2. **DOI matching** - Match using Digital Object Identifier from external IDs
3. **MAG ID matching** - Match using Microsoft Academic Graph identifier

This multi-level approach maximizes recall while maintaining accuracy.

### 4.1 AI Paper Matching

Match Semantic Scholar papers against the OpenAlex AI papera database using the hierarchical matching strategy.

In [ ]:
# ###########################################
#   AI PAPER MATCHING (Title -> DOI -> MAG ID)
#############################################
_tik = time.time()
ai_title_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_ai = TRUE,
    match_method = 'title'
    WHERE EXISTS (
        SELECT 1 FROM ai_db.works AS ai 
        WHERE ai.title = s2.title
    )
""").fetchall()[0][0]
print(f"AI : found {ai_title_match_count} rows by title in {time.time() - _tik:.2f} seconds.")

_tik = time.time()
ai_doi_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_ai = TRUE,
    match_method = 'doi'
    WHERE (is_ai IS NULL OR is_ai = FALSE)
    AND EXISTS (
        SELECT 1 FROM ai_db.works AS ai 
        WHERE ai.doi IS NOT NULL 
        AND ai.doi != ''
        AND json_extract_string(s2.externalids, '$.DOI') = REPLACE(ai.doi, 'https://doi.org/', '')
    )
""").fetchall()[0][0]
print(f"AI : found {ai_doi_match_count} rows by DOI in {time.time() - _tik:.2f} seconds.")

_tik = time.time()
ai_mag_id_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_ai = TRUE,
    match_method = 'mag'
    WHERE (is_ai IS NULL OR is_ai = FALSE)
    AND EXISTS (
        SELECT 1 FROM ai_db.works AS ai 
        WHERE ai.mag_id IS NOT NULL
        AND json_extract_string(s2.externalids, '$.MAG') = CAST(ai.mag_id AS VARCHAR)
    )
""").fetchall()[0][0]
print(f"AI : found {ai_mag_id_match_count} rows by MAG ID in {time.time() - _tik:.2f} seconds.")

# Set is_ai to FALSE for papers that didn't match
cleaned_db.execute("""
    UPDATE papers_with_abstracts 
    SET is_ai = FALSE
    WHERE is_ai IS NULL
""")

print(f"AI matching completed. Found {ai_title_match_count + ai_doi_match_count + ai_mag_id_match_count} AI papers.")

AI : found 499149 rows by title in 1.54 seconds.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

AI : found 26888 rows by DOI in 5.33 seconds.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

AI : found 10525 rows by MAG ID in 4.57 seconds.
AI matching completed. Found 536562 AI papers.


### 4.2 Physics Paper Matching

Match Semantic Scholar papers against the OpenAlex Physics database.

In [20]:
# ###########################################
#   PHYSICS PAPER MATCHING (Title → DOI → MAG ID)
#############################################
_tik = time.time()
physics_title_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_physics = TRUE,
    match_method = 'title'
    WHERE EXISTS (
        SELECT 1 FROM physics_db.works AS physics 
        WHERE physics.title = s2.title
    )
""").fetchall()[0][0]
print(f"Physics : found {physics_title_match_count} rows by title in {time.time() - _tik:.2f} seconds.")

_tik = time.time()
physics_doi_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_physics = TRUE,
    match_method = 'doi'
    WHERE (is_physics IS NULL OR is_physics = FALSE)
    AND EXISTS (
        SELECT 1 FROM physics_db.works AS physics 
        WHERE physics.doi IS NOT NULL 
        AND physics.doi != ''
        AND json_extract_string(s2.externalids, '$.DOI') = REPLACE(physics.doi, 'https://doi.org/', '')
    )
""").fetchall()[0][0]
print(f"Physics : found {physics_doi_match_count} rows by DOI in {time.time() - _tik:.2f} seconds.")

_tik = time.time()
physics_mag_id_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_physics = TRUE,
    match_method = 'mag'
    WHERE (is_physics IS NULL OR is_physics = FALSE)
    AND EXISTS (
        SELECT 1 FROM physics_db.works AS physics 
        WHERE physics.mag_id IS NOT NULL
        AND json_extract_string(s2.externalids, '$.MAG') = CAST(physics.mag_id AS VARCHAR)
    )
""").fetchall()[0][0]
print(f"Physics : found {physics_mag_id_match_count} rows by MAG ID in {time.time() - _tik:.2f} seconds.")

# Set is_physics to FALSE for papers that didn't match
cleaned_db.execute("""
    UPDATE papers_with_abstracts 
    SET is_physics = FALSE
    WHERE is_physics IS NULL
""")    
print(f"Physics matching completed. Found {physics_title_match_count + physics_doi_match_count + physics_mag_id_match_count} Physics papers.")

Physics : found 717764 rows by title in 2.15 seconds.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Physics : found 141167 rows by DOI in 2.95 seconds.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Physics : found 108798 rows by MAG ID in 5.18 seconds.
Physics matching completed. Found 967729 Physics papers.
Physics matching completed. Found 967729 Physics papers.


### 4.3 Psychology Paper Matching

Match Semantic Scholar papers against the OpenAlex Psychology database.

In [21]:
# ###########################################
#   PSYCH PAPER MATCHING (Title → DOI → MAG ID)
#############################################
_tik = time.time()
psych_title_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_psych = TRUE,
    match_method = 'title'
    WHERE EXISTS (
        SELECT 1 FROM psych_db.works AS psych 
        WHERE psych.title = s2.title
    )
""").fetchall()[0][0]
print(f"Psych : found {psych_title_match_count} rows by title in {time.time() - _tik:.2f} seconds.")

_tik = time.time()
psych_doi_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_psych = TRUE,
    match_method = 'doi'
    WHERE (is_psych IS NULL OR is_psych = FALSE)
    AND EXISTS (
        SELECT 1 FROM psych_db.works AS psych 
        WHERE psych.doi IS NOT NULL 
        AND psych.doi != ''
        AND json_extract_string(s2.externalids, '$.DOI') = REPLACE(psych.doi, 'https://doi.org/', '')
    )
""").fetchall()[0][0]
print(f"Psych : found {psych_doi_match_count} rows by DOI in {time.time() - _tik:.2f} seconds.")

_tik = time.time()
psych_mag_id_match_count = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET is_psych = TRUE,    
    match_method = 'mag'
    WHERE (is_psych IS NULL OR is_psych = FALSE)
    AND EXISTS (
        SELECT 1 FROM psych_db.works AS psych 
        WHERE psych.mag_id IS NOT NULL
        AND json_extract_string(s2.externalids, '$.MAG') = CAST(psych.mag_id AS VARCHAR)
    )
""").fetchall()[0][0]
print(f"Psych : found {psych_mag_id_match_count} rows by MAG ID in {time.time() - _tik:.2f} seconds.")

# for psych papers that didn't match, set is_psych to FALSE
cleaned_db.execute("""
    UPDATE papers_with_abstracts 
    SET is_psych = FALSE
    WHERE is_psych IS NULL
""")    
print(f"Psych matching completed. Found {psych_title_match_count + psych_doi_match_count + psych_mag_id_match_count} Psychology papers.")

Psych : found 770461 rows by title in 1.68 seconds.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Psych : found 97313 rows by DOI in 5.02 seconds.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Psych : found 29270 rows by MAG ID in 4.53 seconds.
Psych matching completed. Found 897044 Psychology papers.


### 4.4 Mark Usable Papers

Flag papers that matched at least one domain database as "usable" for downstream analysis.

In [22]:
_tik = time.time()
cleaned_db.execute("""
    UPDATE papers_with_abstracts
    SET usable = TRUE
    WHERE is_physics = TRUE OR is_ai = TRUE OR is_psych = TRUE
""")
print(f"Updated usable flag in {time.time() - _tik:.2f} seconds.")
print(f"Found total usable papers: {cleaned_db.execute('SELECT COUNT(*) FROM papers_with_abstracts WHERE usable = TRUE').fetchone()[0]} ")

Updated usable flag in 0.20 seconds.
Found total usable papers: 2356819 


## 5. Remove Non-Usable Papers

Delete papers that could not be matched to any domain database to reduce dataset size and focus on papers with verified domain classifications.

In [24]:
# Drop non usable rows
_tik = time.time()
cleaned_db.execute("""
    DELETE FROM papers_with_abstracts
    WHERE usable = FALSE or usable IS NULL
""")
print(f"Dropped non-usable rows in {time.time() - _tik:.2f} seconds.")
print(f"Total remaining papers: {cleaned_db.execute('SELECT COUNT(*) FROM papers_with_abstracts').fetchone()[0]}") 

Dropped non-usable rows in 0.45 seconds.
Total remaining papers: 2356819


## 6. Topic Enrichment

Copy the `primary_topic` field from the corresponding OpenAlex domain database for each matched paper. This uses the same matching method (title/DOI/MAG ID) that was used to classify the paper's domain.

In [25]:
# for usable papers, copy the primary topic from corresponding db
_tik = time.time()

# Update primary_topic for AI papers using match_method
ai_topic_title = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = ai.primary_topic
    FROM ai_db.works AS ai
    WHERE s2.is_ai = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'title'
    AND s2.title = ai.title
""").fetchall()[0][0]

ai_topic_doi = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = ai.primary_topic
    FROM ai_db.works AS ai
    WHERE s2.is_ai = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'doi'
    AND s2.primary_topic IS NULL
    AND ai.doi IS NOT NULL 
    AND ai.doi != ''
    AND json_extract_string(s2.externalids, '$.DOI') = REPLACE(ai.doi, 'https://doi.org/', '')
""").fetchall()[0][0]

ai_topic_mag = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = ai.primary_topic
    FROM ai_db.works AS ai
    WHERE s2.is_ai = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'mag'
    AND s2.primary_topic IS NULL
    AND ai.mag_id IS NOT NULL
    AND json_extract_string(s2.externalids, '$.MAG') = CAST(ai.mag_id AS VARCHAR)
""").fetchall()[0][0]

ai_topic_count = ai_topic_title + ai_topic_doi + ai_topic_mag
print(f"Updated {ai_topic_count} AI paper topics ({ai_topic_title} title, {ai_topic_doi} doi, {ai_topic_mag} mag) in {time.time() - _tik:.2f} seconds.")

# Update primary_topic for Physics papers using match_method
_tik = time.time()
physics_topic_title = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = physics.primary_topic
    FROM physics_db.works AS physics
    WHERE s2.is_physics = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'title'
    AND s2.primary_topic IS NULL
    AND s2.title = physics.title
""").fetchall()[0][0]

physics_topic_doi = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = physics.primary_topic
    FROM physics_db.works AS physics
    WHERE s2.is_physics = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'doi'
    AND s2.primary_topic IS NULL
    AND physics.doi IS NOT NULL 
    AND physics.doi != ''
    AND json_extract_string(s2.externalids, '$.DOI') = REPLACE(physics.doi, 'https://doi.org/', '')
""").fetchall()[0][0]

physics_topic_mag = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = physics.primary_topic
    FROM physics_db.works AS physics
    WHERE s2.is_physics = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'mag'
    AND s2.primary_topic IS NULL
    AND physics.mag_id IS NOT NULL
    AND json_extract_string(s2.externalids, '$.MAG') = CAST(physics.mag_id AS VARCHAR)
""").fetchall()[0][0]

physics_topic_count = physics_topic_title + physics_topic_doi + physics_topic_mag
print(f"Updated {physics_topic_count} Physics paper topics ({physics_topic_title} title, {physics_topic_doi} doi, {physics_topic_mag} mag) in {time.time() - _tik:.2f} seconds.")

# Update primary_topic for Psychology papers using match_method
_tik = time.time()
psych_topic_title = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = psych.primary_topic
    FROM psych_db.works AS psych
    WHERE s2.is_psych = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'title'
    AND s2.primary_topic IS NULL
    AND s2.title = psych.title
""").fetchall()[0][0]

psych_topic_doi = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = psych.primary_topic
    FROM psych_db.works AS psych
    WHERE s2.is_psych = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'doi'
    AND s2.primary_topic IS NULL
    AND psych.doi IS NOT NULL 
    AND psych.doi != ''
    AND json_extract_string(s2.externalids, '$.DOI') = REPLACE(psych.doi, 'https://doi.org/', '')
""").fetchall()[0][0]

psych_topic_mag = cleaned_db.execute("""
    UPDATE papers_with_abstracts AS s2
    SET primary_topic = psych.primary_topic
    FROM psych_db.works AS psych
    WHERE s2.is_psych = TRUE
    AND s2.usable = TRUE
    AND s2.match_method = 'mag'
    AND s2.primary_topic IS NULL
    AND psych.mag_id IS NOT NULL
    AND json_extract_string(s2.externalids, '$.MAG') = CAST(psych.mag_id AS VARCHAR)
""").fetchall()[0][0]

psych_topic_count = psych_topic_title + psych_topic_doi + psych_topic_mag
print(f"Updated {psych_topic_count} Psychology paper topics ({psych_topic_title} title, {psych_topic_doi} doi, {psych_topic_mag} mag) in {time.time() - _tik:.2f} seconds.")

print(f"Total topics updated: {ai_topic_count + physics_topic_count + psych_topic_count}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Updated 1024219 AI paper topics (986986 title, 26828 doi, 10405 mag) in 29.73 seconds.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Updated 1001656 Physics paper topics (751382 title, 141477 doi, 108797 mag) in 6.52 seconds.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Updated 888438 Psychology paper topics (765472 title, 94718 doi, 28248 mag) in 9.22 seconds.
Total topics updated: 2914313


## 7. Data Validation

Verify the cleaned database by examining sample records and checking influential papers in each domain.

In [18]:
# sample of final cleaned db
cleaned_db.execute("SELECT * FROM papers_with_abstracts LIMIT 5").df()

,corpusid,title,abstract,publication_date,citation_count,influential_citation_count,primary_topic,is_physics,is_ai,is_psych
0,214745815,Using Expectancy Violations Theory to Understa...,As robots are increasingly placed in direct in...,2020-03-23,10,0,Psychology of Social Influence,False,False,True
1,119306079,Rapidity scaling of multiplicity and flow in w...,We examine the 'naturalness' of the scaling of...,2009-11-25,11,0,High-Energy Particle Collisions Research,True,False,False
2,234399577,Mining High Utility Itemsets Based on Pattern ...,Mining high utility itemsets (HUIs) has been a...,2020-12-25,9,0,Data Mining Algorithms and Applications,False,True,False
3,123487525,First Principles Study of Cerium Monochalcogen...,"A theoretical investigation of structural, mag...",2014-07-21,0,0,Rare-earth and actinide compounds,True,False,False
4,18695655,From boundary to bulk in logarithmic CFT,The analogue of the charge-conjugation modular...,2007-07-03,110,7,Black Holes and Theoretical Physics,True,False,False


In [4]:
# most influential papers in ai
cleaned_db.execute("SELECT * FROM papers_with_abstracts WHERE is_ai = TRUE ORDER BY influential_citation_count DESC LIMIT 10").df()

,corpusid,title,abstract,externalids,publication_date,citation_count,influential_citation_count,usable,primary_topic,is_physics,is_ai,is_psych,match_method
0,206594692,Deep Residual Learning for Image Recognition,Deeper neural networks are more difficult to t...,"{""Medline"": null, ""MAG"": ""2194775991"", ""ACL"": ...",2015-12-10,212958,31078,True,Advanced Neural Network Applications,False,True,False,title
1,14124313,Very Deep Convolutional Networks for Large-Sca...,In this work we investigate the effect of the ...,"{""Medline"": null, ""MAG"": ""1686810756"", ""ACL"": ...",2014-09-04,107085,14296,True,Advanced Vision and Imaging,False,True,False,title
2,10328909,Faster R-CNN: Towards Real-Time Object Detecti...,State-of-the-art object detection networks dep...,"{""Medline"": ""27295650v1"", ""MAG"": ""639708223"", ...",2015-06-04,68158,9460,True,Advanced Neural Network Applications,False,True,False,title
3,225039882,An Image is Worth 16x16 Words: Transformers fo...,While the Transformer architecture has become ...,"{""Medline"": null, ""MAG"": ""3094502228"", ""ACL"": ...",2020-10-22,52559,6208,True,Advanced Neural Network Applications,False,True,False,title
4,2930547,ImageNet Large Scale Visual Recognition Challenge,The ImageNet Large Scale Visual Recognition Ch...,"{""Medline"": null, ""MAG"": ""2117539524"", ""ACL"": ...",2014-09-01,41212,4884,True,Image Retrieval and Classification Techniques,False,True,False,title
5,245335280,High-Resolution Image Synthesis with Latent Di...,By decomposing the image formation process int...,"{""Medline"": null, ""MAG"": null, ""ACL"": null, ""D...",2021-12-20,19972,4703,True,Generative Adversarial Networks and Image Synt...,False,True,False,title
6,206592484,Going deeper with convolutions,We propose a deep convolutional neural network...,"{""Medline"": null, ""MAG"": ""2950179405"", ""ACL"": ...",2014-09-16,45792,4358,True,Advanced Neural Network Applications,False,True,False,title
7,54465873,Mask R-CNN,"We present a conceptually simple, flexible, an...","{""Medline"": null, ""MAG"": null, ""ACL"": null, ""D...",2017-03-20,29935,4145,True,Advanced Neural Network Applications,False,True,False,title
8,1957433,GloVe: Global Vectors for Word Representation,Recent methods for learning vector space repre...,"{""Medline"": null, ""MAG"": ""2250539671"", ""ACL"": ...",2014-10-01,33517,3991,True,Topic Modeling,False,True,False,mag
9,4650265,XGBoost: A Scalable Tree Boosting System,Tree boosting is a highly effective and widely...,"{""Medline"": null, ""MAG"": ""2295598076"", ""ACL"": ...",2016-03-09,45470,3599,True,Machine Learning and Data Classification,False,True,False,doi


## 8. Schema Cleanup and Optimization

Remove intermediate columns (`externalids`, `usable`, `match_method`) that are no longer needed, then vacuum the database to reclaim storage space.

In [6]:
#
COLUMNS_TO_DROP = ['externalids', 'usable', 'match_method']

for col in COLUMNS_TO_DROP:
    cleaned_db.execute(f"ALTER TABLE papers_with_abstracts DROP COLUMN {col}")


In [20]:
# detach dbs
# cleaned_db.execute("DETACH physics_db;")
# cleaned_db.execute("DETACH ai_db;")
# cleaned_db.execute("DETACH psych_db;")
# cleaned_db.execute("DETACH papers_w_abstracts_db;")

# Reclaim space from deleted rows and optimize database
_tik = time.time()
cleaned_db.execute("VACUUM")
print(f"VACUUM completed in {time.time() - _tik:.2f} seconds.")

VACUUM completed in 0.00 seconds.


## 9. Storage Optimization via Table Rebuild

Create an optimized copy of the database by rebuilding the table from scratch. This eliminates fragmentation from update/delete operations and achieves optimal storage compression.

In [26]:
# Rebuild the table to optimize storage
_tik = time.time()
temp_db = duckdb.connect(os.path.join(DB_DIR, 'S2_papers_cleaned_optimized.db'))

# Attach the original cleaned database and copy data with a fresh CREATE TABLE AS SELECT
temp_db.execute(f"ATTACH '{os.path.join(DB_DIR, CLEANED_UP_S2_PAPERS_DB_FILENAME)}' AS source_db")
temp_db.execute("""
    CREATE TABLE papers_with_abstracts AS 
    SELECT * FROM source_db.papers_with_abstracts
""")
temp_db.execute("DETACH source_db")
temp_db.close()

# Check new size
optimized_size = os.path.getsize(os.path.join(DB_DIR, 'S2_papers_cleaned_optimized.db'))
original_cleaned_size = os.path.getsize(os.path.join(DB_DIR, CLEANED_UP_S2_PAPERS_DB_FILENAME))

print(f"Rebuilding completed in {time.time() - _tik:.2f} seconds.")
print(f"\nOriginal cleaned DB: {original_cleaned_size / (1024**2):.2f} MB")
print(f"Optimized DB:        {optimized_size / (1024**2):.2f} MB")
print(f"Space saved:         {(original_cleaned_size - optimized_size) / (1024**2):.2f} MB ({(1 - optimized_size/original_cleaned_size)*100:.1f}%)")
print(f"Bytes per row:       {optimized_size / 2356819:.0f}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rebuilding completed in 13.42 seconds.

Original cleaned DB: 17280.51 MB
Optimized DB:        4118.51 MB
Space saved:         13162.00 MB (76.2%)
Bytes per row:       1832


## 10. Final Verification

Verify the optimized database contents and examine top papers by domain to ensure data integrity.

In [27]:
# Verify optimized database contents
optimized_db = duckdb.connect(os.path.join(DB_DIR, 'S2_papers_cleaned_optimized.db'))

# Check row count
row_count = optimized_db.execute("SELECT COUNT(*) FROM papers_with_abstracts").fetchone()[0]
print(f"Total rows in optimized DB: {row_count:,}")

# Sample some rows
print("\nSample rows:")
optimized_db.execute("SELECT * FROM papers_with_abstracts LIMIT 5").df()

Total rows in optimized DB: 2,356,819

Sample rows:


,corpusid,title,abstract,publication_date,citation_count,influential_citation_count,primary_topic,is_physics,is_ai,is_psych
0,214745815,Using Expectancy Violations Theory to Understa...,As robots are increasingly placed in direct in...,2020-03-23,10,0,Psychology of Social Influence,False,False,True
1,119306079,Rapidity scaling of multiplicity and flow in w...,We examine the 'naturalness' of the scaling of...,2009-11-25,11,0,High-Energy Particle Collisions Research,True,False,False
2,234399577,Mining High Utility Itemsets Based on Pattern ...,Mining high utility itemsets (HUIs) has been a...,2020-12-25,9,0,Data Mining Algorithms and Applications,False,True,False
3,123487525,First Principles Study of Cerium Monochalcogen...,"A theoretical investigation of structural, mag...",2014-07-21,0,0,Rare-earth and actinide compounds,True,False,False
4,18695655,From boundary to bulk in logarithmic CFT,The analogue of the charge-conjugation modular...,2007-07-03,110,7,Black Holes and Theoretical Physics,True,False,False


In [28]:
# most influential papers in ai
print("\nMost influential AI papers:")
optimized_db.execute("SELECT * FROM papers_with_abstracts WHERE is_ai = TRUE ORDER BY influential_citation_count DESC LIMIT 10").df()


Most influential AI papers:


,corpusid,title,abstract,publication_date,citation_count,influential_citation_count,primary_topic,is_physics,is_ai,is_psych
0,206594692,Deep Residual Learning for Image Recognition,Deeper neural networks are more difficult to t...,2015-12-10,212958,31078,Advanced Neural Network Applications,False,True,False
1,14124313,Very Deep Convolutional Networks for Large-Sca...,In this work we investigate the effect of the ...,2014-09-04,107085,14296,Advanced Vision and Imaging,False,True,False
2,10328909,Faster R-CNN: Towards Real-Time Object Detecti...,State-of-the-art object detection networks dep...,2015-06-04,68158,9460,Advanced Neural Network Applications,False,True,False
3,225039882,An Image is Worth 16x16 Words: Transformers fo...,While the Transformer architecture has become ...,2020-10-22,52559,6208,Advanced Neural Network Applications,False,True,False
4,2930547,ImageNet Large Scale Visual Recognition Challenge,The ImageNet Large Scale Visual Recognition Ch...,2014-09-01,41212,4884,Image Retrieval and Classification Techniques,False,True,False
5,245335280,High-Resolution Image Synthesis with Latent Di...,By decomposing the image formation process int...,2021-12-20,19972,4703,Generative Adversarial Networks and Image Synt...,False,True,False
6,206592484,Going deeper with convolutions,We propose a deep convolutional neural network...,2014-09-16,45792,4358,Advanced Neural Network Applications,False,True,False
7,54465873,Mask R-CNN,"We present a conceptually simple, flexible, an...",2017-03-20,29935,4145,Advanced Neural Network Applications,False,True,False
8,1957433,GloVe: Global Vectors for Word Representation,Recent methods for learning vector space repre...,2014-10-01,33517,3991,Topic Modeling,False,True,False
9,4650265,XGBoost: A Scalable Tree Boosting System,Tree boosting is a highly effective and widely...,2016-03-09,45470,3599,Machine Learning and Data Classification,False,True,False


In [29]:
optimized_db.close()

## 11. Upload to Hugging Face Hub

Upload the optimized database to Hugging Face Hub for public access and reproducibility.

In [ ]:

from huggingface_hub import login, upload_folder, upload_file, file_download, hf_hub_download

In [ ]:
login(token='') #Hugging Face token

In [ ]:
upload_file(
    path_or_fileobj='db/S2_papers_cleaned_optimized.db', 
    repo_id="lalit3c/S2_CS_PHY_PYSCH_papers",
    repo_type="dataset",  # specify it's a dataset repo
    path_in_repo='S2_papers_cleaned.db' 
)